<a href="https://colab.research.google.com/github/huuquangnhatnguyen/RoboticDataVisual/blob/main/SideViewModelforKuka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This function is used to load the original MATLAB file of information into python and transform it into a dictionary with 10 keys respectively according to 10 trajectories. 




In [ ]:
from scipy.io import loadmat
# loading matlab file and split every row 
import pandas as pd
def loadingMatfiles(filename):
  rawDict = loadmat(filename)
  keys = []
  for i in range (1,11):
    sampleName = ("baxtertraj" if filename == "BaxterDirectDynamics.mat" else "kukatraj") + str(i)
    keys.append(sampleName)
  samples = []
  for key in keys:
    samples.append(pd.DataFrame(rawDict[key]))
  return samples
  



This function is used to split the original dataframe into smaller dataframes, which contain the information of position, velocity and torque of the 7 joints. 

In [ ]:
# this function is used to split the original 

def ColumnSplit(trajectory):
  SensorPosition = trajectory.iloc[ : , [0,1,2,3,4,5,6]].copy()
  SensorVelo = trajectory.iloc[:, [7,8,9,10,11,12,13]].copy()
  SensorTorques = trajectory.iloc[:, [14,15,16,17,18,19,20]].copy()
  TargetPosition = trajectory.iloc[:, [21,22,23,24,25,26,27]].copy()
  TargetVelo = trajectory.iloc[:, [28,29,30,31,32,33,34]].copy()
  Iterdf = {"SensorPosition" : SensorPosition,
          "SensorVelo":SensorVelo,
          "SensorTorques":SensorTorques,
          "TargetPosition":TargetPosition,
          "TargetVelo": TargetVelo}

  # change the name of the column in side the dataframe into joint #number

  Iterdf["SensorPosition"].rename(columns = lambda s: str(s + 1) ,inplace=True)
  Iterdf["SensorPosition"].rename(columns = lambda s: "Joint " + s ,inplace=True)
  
  Iterdf["SensorVelo"].rename(columns = lambda s: str(s - 6) ,inplace=True)
  Iterdf["SensorVelo"].rename(columns = lambda s: "Joint " + s ,inplace=True)
  

  Iterdf["SensorTorques"].rename(columns = lambda s: str(s - 13) ,inplace=True)
  Iterdf["SensorTorques"].rename(columns = lambda s: "Joint " + s ,inplace=True)
  
  Iterdf["TargetPosition"].rename(columns = lambda s: str(s - 20) ,inplace=True)
  Iterdf["TargetPosition"].rename(columns = lambda s: "Joint " + s ,inplace=True)
  
  Iterdf["TargetVelo"].rename(columns = lambda s: str(s - 27) ,inplace=True)
  Iterdf["TargetVelo"].rename(columns = lambda s: "Joint " + s ,inplace=True)
  
  
  return Iterdf





Run the code block below to conduct spliting the original dataframe input from the MATLAB file into dataframes inside a dictionary with keywords

In [ ]:
kuka = loadingMatfiles("KukaDirectDynamics.mat")
ColumnSplit(kuka[5])

With the dictionary above, I will plot out line plots to describe the tranformation in positions, velocity and torque of the robot over iterations of the pick and place process


In [ ]:
import os
import matplotlib.pyplot as plt
path = "/content/plot"
os.mkdir(path)

def linePlot(robot):
  traj = 0
  for robottraj in robot:
    traj = traj + 1
    curTraj = ColumnSplit(robottraj)
    
    #directory = "robottraj" + traj
    #os.mkdir(directory)
    for aspect in curTraj.keys():
      filename = "/content/plot/" + (aspect  + str(traj) + ".png")
      curTraj[aspect].plot(subplots = True, title = aspect)
      plt.savefig(filename,facecolor = "w", edgecolor='w', transparent = False)






This below code is used to run and visualize the data from the dataframe of the Kuka Robots

In [ ]:
kuka = loadingMatfiles("KukaDirectDynamics.mat")
linePlot(kuka)

This code is used to create a folder to store a visualization of of the motion of Kuka

In [ ]:
import os
path = "/content/figure"
os.mkdir(path)

This below code is to create graph of motion of every iteration and generate a gif according to the series of images and changes 

In [ ]:
import imageio
import math
import matplotlib.pyplot as plt
import numpy as np

def motionModelSide(kukatraj,path): 

  sideView = kukatraj.iloc[:, [1,3,5]].copy()

  l1 = 40
  l2 = 40
  l3 = 16

  figno = 1
  for i in range(0,len(sideView.index)):
    x1 = l1 * math.cos(sideView.iloc[i][1])
    y1 = l1 * math.sin(sideView.iloc[i][1])
    x2 = x1 + l2 * math.cos(sideView.iloc[i][3])
    y2 = y1 + l2 * math.sin(sideView.iloc[i][3])
    x3 = x2 + l3 * math.cos(sideView.iloc[i][5])
    y3 = y2 + l3 * math.sin(sideView.iloc[i][5])

    filename = path + "/" + ("frame" + str(figno) + '.jpg')
    figno = figno + 1

    plt.figure()
    plt.plot([0,x1],[34,y1])
    plt.plot([x1,x2],[y1,y2])
    plt.plot([x2,x3],[y2,y3])
    plt.xlim([0,120])
    plt.ylim([-120,120])
    plt.savefig(filename)

  images=[]
  for i in range(1,len(sideView.index)):
    filename = path + "/" + "frame" + str(i)+".jpg"
    images.append(imageio.imread(filename))
  imageio.mimsave('/content/figure/Traj.gif', images, fps = 30)



Run the block of code below to process log out the motion model of a trajectory

In [ ]:
import os
path = "/content/KukaTraj"
os.mkdir(path)
kuka = loadingMatfiles("KukaDirectDynamics.mat")
traj1 = kuka[7]         # replace index with the designated one, no of trajectory = index + 1
motionModelSide(traj1,path)

This is a demo testing for baxter right arm side view

In [ ]:
import os
import imageio
import math
import matplotlib.pyplot as plt
import numpy as np

def motionModelSide(baxtertraj,path): 

  sideView = baxtertraj.iloc[:, [1,3,5]].copy()

  l1 = 7
  l2 = 1
  l3 = 16

  figno = 1
  for i in range(0,len(sideView.index)):
    x1 = l1 * math.cos(sideView.iloc[i][1])
    y1 = l1 * math.sin(sideView.iloc[i][1])
    x2 = x1 + l2 * math.cos(sideView.iloc[i][3])
    y2 = y1 + l2 * math.sin(sideView.iloc[i][3])
    x3 = x2 + l3 * math.cos(sideView.iloc[i][5])
    y3 = y2 + l3 * math.sin(sideView.iloc[i][5])

    filename = path + "/" + ("frame" + str(figno) + '.jpg')
    figno = figno + 1

    plt.figure()
    plt.plot([0,x1],[27,y1])
    plt.plot([x1,x2],[y1,y2])
    plt.plot([x2,x3],[y2,y3])
    plt.xlim([0,120])
    plt.ylim([0,120])
    plt.savefig(filename)

  images=[]
  for i in range(1,len(sideView.index)):
    filename = path + "/" + "frame" + str(i)+".jpg"
    images.append(imageio.imread(filename))
  imageio.mimsave('/content/figure/moviebdemo.gif', images, fps = 30)

